In [1]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import pandas as pd
import re

File fiverr_data.csv disimpan di folder tmp jika menggunakan colab. Jika running notebook secara lokal gunakan direktori lokal.

In [2]:
fiverr_scraped = pd.read_csv('/tmp/fiverr_data.csv', delimiter=',')

regex digunakan untuk mengekstrak data negara asal ilustrator dan kecepatan respon. String yang di input ke `re.search()` memiliki bentuk umum seperti: FromPakistanMember sinceJul 2019Avg. response time1 hourLast deliveryabout 9 hoursLanguagesEnglish

In [3]:
regex = r"From(.*)Member.*time(\d)"

In [30]:
illustrator_origins=[]
illustrator_response_times=[]
client_origins=[]
client_names=[]

Sintaks dibawah ini adalah sintaks web scraping per halaman gig di Fiverr. HATI-HATI: jangan scraping lebih dari 5 URL per iterasi karena akan di blokir oleh Fiverr dan tidak bisa scraping lagi.

In [31]:
for url in fiverr_scraped['gig_url'][:5]#masukkan index url (sekali scraping maksimal 5)
    req = Request(url, None, headers={'User-Agent':'Safari/537.11'})
    webpage = urlopen(req).read()
    soup = BeautifulSoup(webpage, 'html.parser')
    stats_containers = soup.find_all('div', class_='stats-desc')
    review_containers = soup.find_all('ul', class_='review-list')
    for stats_container in stats_containers:
        illustrator_stat = stats_container.find('ul', class_='user-stats').text.strip()
        result = re.search(regex,illustrator_stat)
        illustrator_origin=result[1]
        illustrator_response_time=result[2]
        illustrator_origins.append(illustrator_origin)
        illustrator_response_times.append(illustrator_response_time)
    for review_container in review_containers:
        country = review_container.find('div', class_='country-name tbody-6').text.strip()
        client_name = review_container.find('span', class_='tbody-5 co-grey-1100 username').text.strip()
    client_origins.append(country)
    client_names.append(client_name)

hasil scraping akan terlihat seperti berikut:  
```
['Pakistan', 'Bangladesh', 'India', 'Pakistan', 'Sri Lanka']
['1', '1', '1', '1', '4']
['stoxventures', 'lifehossain', 'stormproofrooof', 'remi_bgd', 'tribolumen']
['Canada', 'Italy', 'United States', 'France', 'United States']
```
periksa setiap list. Semuanya harus memiliki panjang (`length`) yang sama


In [ ]:
#periksa hasil scraping
print(illustrator_origins)
print(illustrator_response_times)
print(client_names)
print(client_origins)

Simpan list dalam pandas dataframe

In [33]:
fiverr_df = pd.DataFrame(list(zip(client_origins, client_names, illustrator_origins, illustrator_response_times)),
                         columns = ['client_origins', 'client_names', 'illustrator_origins', 'illustrator_response_times'])

Simpan dataframe dalam file csv di dalam folder tmp (atau gunakan direktori lokal)

In [34]:
fiverr_df.to_csv('/tmp/fiverr_df.csv')